In [13]:
import os
from feast import Client, Feature, Entity, ValueType, FeatureTable
from feast.feature import SQLFeature
from feast.data_source import FileSource, KafkaSource
from feast.data_format import ParquetFormat, AvroFormat

client = Client()
print(client.version())
print(client.list_feature_tables())


/home/tzhu/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
{'sdk': {'version': '0.9.0rc2.dev33+g4376259d.d20210216'}, 'serving': 'not configured', 'core': {'url': 'localhost:6565', 'version': '0.10.0-SNAPSHOT'}}
[]


In [14]:
feature1 = SQLFeature(name='last_7_day_total_trips', query="select driver_id, count(*) over (partition by driver_id order by ts range between interval '7' day preceding and current row) from driver_stats")
feature2 = SQLFeature(name='last_7_day_avg_amount', query="select driver_id, avg(amount) over (partition by driver_id order by ts range between interval '7' day preceding and current row) from driver_stats")

driver_id = Entity(name="driver_id", description="Driver identifier", value_type=ValueType.INT64)
client.apply(driver_id)

feature_table = FeatureTable(
    name = "driver_statistics",
    entities = ["driver_id"],
    features = [
        feature1,
        feature2,
    ],
    batch_source=FileSource(
        event_timestamp_column="ts",
        file_format=ParquetFormat(),
        file_url='/home/tzhu/data/test_data',
    )
)


client.apply_feature_table(feature_table)

# offline_features = client.getOffline(feature_table)
# online_feature = clinet.getOnline(feature_table, entity_id='1234')

/home/tzhu/works/feast/sdk/python/feast/client.py:644: DeprecationWarning: The method apply_feature_table() is being deprecated. Please use apply() instead. Feast 0.10 and onwards will not support apply_feature_table().
  warnings.warn(


In [15]:
print(client.list_feature_tables())
print(client.list_entities())

[FeatureTable <driver_statistics>]
